In [19]:
# Installation des dépendances
# pip install pip
# pip install azure-ai-textanalytics


In [20]:
#Authentification aux services et import des librairies interfacant Azure Cognitive Services
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

credential = AzureKeyCredential("<KEY>")
text_analytics_client = TextAnalyticsClient(endpoint="https://<ENDPOINT>.cognitiveservices.azure.com/", credential=credential)

In [21]:
#lecture des fichiers paragraphes a analyser/langues des paragraphes dans un DataFrame
import pandas as pd
test = pd.read_csv("dataset/x_test.txt", header=None, delimiter="\r\n",engine='python', names=['text'], encoding="utf-8")
test['label'] = pd.read_table("dataset/y_test.txt", header=None, keep_default_na=False)
print(test.columns.values)
print("Dimensions de la DataFrame:  ", test.shape)

['text' 'label']
Dimensions de la DataFrame:   (117500, 2)


In [22]:
#lecture du tableau de correspondances des labels langues
data = pd.read_csv("dataset/labels.csv", sep=';')

In [36]:
# definition d'une fonction de correspondances entre labels et labels ISO
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

def getISO3693lang(value):
    if value=="zh_chs" or value=="zh_cht" :
        return "zho"
    else :
        return data['ISO 369-3'][getIndexes(data, value)[0][0]]

In [27]:
# creation d'un subset de paragraphes avec les 5 langues les plus parlees au monde
num_paragraph = 100

test_reduce = test.loc[test['label'] == getISO3693lang('en')].head(num_paragraph)
test_reduce = test_reduce.append(test.loc[test['label'] == getISO3693lang('zh')].head(num_paragraph), ignore_index=True, sort=False)
# test_reduce = test_reduce.append(test.loc[test['label'] == getISO3693lang('hi')].head(num_paragraph), sort=False)
test_reduce = test_reduce.append(test.loc[test['label'] == getISO3693lang('hi')].head(num_paragraph), ignore_index=True, sort=False)
test_reduce = test_reduce.append(test.loc[test['label'] == getISO3693lang('es')].head(num_paragraph), ignore_index=True, sort=False)
test_reduce = test_reduce.append(test.loc[test['label'] == getISO3693lang('ar')].head(num_paragraph), ignore_index=True, sort=False)

In [28]:
#Analyse par l'API Cognitive Services. On s'assure qu'aucun resultat retourne est en erreur
response = text_analytics_client.detect_language(test_reduce.text.tolist())

#HttpResponseError: (PayloadTooLarge) Max allowed payload is 1MB. For details see https://aka.ms/text-analytics-data-limits
# response = text_analytics_client.detect_language(test.text.tolist())

print("Dimensionnalité des resultats:  ", len(response))
result = [doc for doc in response if not doc.is_error]
print("Dimensionnalité des resultats filtres:  ", len(result))

Dimensionnalité des resultats:   500
Dimensionnalité des resultats filtres:   500


In [40]:
# Visualisation des resultats
i=0
for doc in result:
    print("Language to test : {} || Language returned by MS : {}  ".format(test_reduce['label'].iloc[i], getISO3693lang(doc.primary_language.iso6391_name)  ))
     if test_reduce['label'].iloc[i] != getISO3693lang(doc.primary_language.iso6391_name) :
        
    i += 1

Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to test : eng || Language returned by MS : eng  
Language to te

In [44]:
# Visualisation des paragraphes en erreur
i=0
for doc in result:
    if test_reduce['label'].iloc[i] != getISO3693lang(doc.primary_language.iso6391_name) :
        print("Language to test : {} || Language returned by MS : {}  ".format(test_reduce['label'].iloc[i], getISO3693lang(doc.primary_language.iso6391_name)  ))
        print(test_reduce['text'].iloc[i])
    i += 1

Language to test : spa || Language returned by MS : eng  
Bandyopadhyay, S. & RoyChowdhury, K. 1996. Beginning of the Continental Jurassic in India: A Paleontological Approach. Museum of Northern Arizona Bulletin. The Continental Jurassic. Transactions of the Continental Jurassic Symposium. October 21-23, M. Morales (ed.). Museum of Northern Arizona60, 371-378.
Language to test : spa || Language returned by MS : eng  
Yadagiri, P. 1988. A new sauropod Kotasaurus yamanpalliensis from Lower Jurassic Kota Formation of India. Records of the Geological Survey of India, 11, 102-127.


In [ ]:
# Nous nous apercevons que les paragraphes sont bien en anglais. Ils peuvent correspondre a des titres de livres ecrits en anglais dans un contexte espagnol.